In [11]:
import pandas as pd
df = pd.read_excel("Flow_depth.xls")
df.head()

,IDs:,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100190,92100220,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1
0,Date/Time,Depth,Depth,Depth,Depth,Depth,Depth,Depth,Depth,Depth,...,Depth,Depth,Depth,Depth,Depth,Depth,Depth,Depth,Depth,Depth
1,M/d/yyyy,ft,ft,ft,ft,ft,ft,ft,ft,ft,...,ft,ft,ft,ft,ft,ft,ft,ft,ft,ft
2,2023-10-01 00:10:00,0.124174,0.06161,0.065682,0.070087,0.095688,0.058608,0.062742,0.06277,0.058495,...,0.024923,0.078109,0.124467,0.091907,0.064839,0.039564,0.12718,0.102981,0.116926,0.048051
3,2023-10-01 00:20:00,0.195642,0.061608,0.126653,0.081124,0.144584,0.128025,0.057909,0.094791,0.057865,...,0.03182,0.155917,0.17709,0.141373,0.146733,0.100651,0.140623,0.162597,0.117399,0.08378
4,2023-10-01 00:30:00,0.264221,0.061608,0.183327,0.131465,0.200239,0.191842,0.057831,0.108786,0.066044,...,0.032719,0.198622,0.216264,0.17466,0.193832,0.150146,0.180615,0.171752,0.117279,0.11803


In [12]:
df = df.iloc[2:].reset_index(drop=True)
df = df.rename(columns = {'IDs:':'Date/Time'})
df.head()

,Date/Time,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100190,92100220,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1
0,2023-10-01 00:10:00,0.124174,0.06161,0.065682,0.070087,0.095688,0.058608,0.062742,0.06277,0.058495,...,0.024923,0.078109,0.124467,0.091907,0.064839,0.039564,0.12718,0.102981,0.116926,0.048051
1,2023-10-01 00:20:00,0.195642,0.061608,0.126653,0.081124,0.144584,0.128025,0.057909,0.094791,0.057865,...,0.03182,0.155917,0.17709,0.141373,0.146733,0.100651,0.140623,0.162597,0.117399,0.08378
2,2023-10-01 00:30:00,0.264221,0.061608,0.183327,0.131465,0.200239,0.191842,0.057831,0.108786,0.066044,...,0.032719,0.198622,0.216264,0.17466,0.193832,0.150146,0.180615,0.171752,0.117279,0.11803
3,2023-10-01 00:40:00,0.331028,0.061608,0.242019,0.173434,0.242876,0.233405,0.05783,0.108009,0.065615,...,0.032861,0.233713,0.248647,0.203819,0.231262,0.166831,0.200379,0.163299,0.117266,0.151517
4,2023-10-01 00:50:00,0.380019,0.061608,0.287735,0.201139,0.270772,0.257954,0.05783,0.107841,0.065494,...,0.032884,0.243812,0.260042,0.209928,0.24409,0.165327,0.201782,0.159117,0.117265,0.176059


In [2]:
df.shape

(17706, 24)

In [3]:
import pandas as pd
import numpy as np

# --- assume df is your DataFrame and the datetime column is named 'Date/Time' ---
# If your datetime column has a different name, change it below.
df = df.copy()
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

# create year/month/half indicator (half = 1 for 1..15, 2 for 16..end)
df['year']  = df['Date/Time'].dt.year
df['month'] = df['Date/Time'].dt.month
df['half']  = np.where(df['Date/Time'].dt.day <= 15, 1, 2)

# assign a sequential half-month id (1,2,3...) in chronological order
# groupby(..., sort=True).ngroup() will produce 0-based group numbers in sorted order
df['period_id'] = df.groupby(['year','month','half'], sort=True).ngroup() + 1
df['period'] = (
    df['year'].astype(str) + "-" +
    df['month'].astype(str).str.zfill(2) + "-" +
    df['half'].astype(str)
)

# choose the node columns to aggregate (everything except helper columns)
exclude = ['Date/Time','year','month','half','period_id','period']
node_cols = [c for c in df.columns if c not in exclude]

# --- AGGREGATION OPTION A: SUM (default) ---
agg_sum = df.groupby('period', sort=True)[node_cols].mean().reset_index()

# If you want a meaningful representative datetime for each period, add:
# create a mapping from period -> (period_start, period_end) using the earliest/largest timestamp
period_bounds = (df.groupby('period', sort=True)
                   .agg(period_start=('Date/Time','min'),
                        period_end  =('Date/Time','max'))
                   .reset_index())
agg_sum = agg_sum.merge(period_bounds, on='period')

# --- AGGREGATION OPTION B: MEAN (if you prefer averages instead of sums) ---
agg_mean = df.groupby('period', sort=True)[node_cols].mean().reset_index()
agg_mean = agg_mean.merge(period_bounds, on='period')

agg_sum.head()

,period,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1,period_start,period_end
0,2023-10-1,0.518002,0.167731,0.434051,0.344125,0.418058,0.407859,0.103951,0.201653,0.167112,...,0.409344,0.370781,0.393104,0.33826,0.381042,0.289327,0.209857,0.221243,2023-10-01 00:10:00,2023-10-15 23:50:00
1,2023-10-2,0.431114,0.081366,0.335041,0.225448,0.295111,0.273384,0.060936,0.11328,0.07063,...,0.270609,0.217378,0.254522,0.170378,0.208554,0.164932,0.122078,0.201075,2023-10-16 00:00:00,2023-10-31 23:50:00
2,2023-11-1,0.496227,0.144761,0.407513,0.2966,0.36626,0.339725,0.06881,0.135149,0.091894,...,0.334766,0.276567,0.315738,0.21648,0.261223,0.185193,0.1327,0.222163,2023-11-01 00:00:00,2023-11-15 23:50:00
3,2023-11-2,0.421983,0.073063,0.325788,0.219035,0.288693,0.267365,0.059269,0.110441,0.067971,...,0.264707,0.212423,0.248855,0.166509,0.203932,0.161341,0.119923,0.196774,2023-11-16 00:00:00,2023-11-30 23:50:00
4,2023-12-1,0.500641,0.151655,0.416197,0.343451,0.413081,0.422506,0.150682,0.253342,0.210518,...,0.431369,0.398949,0.420956,0.367641,0.407908,0.325668,0.249231,0.210647,2023-12-01 00:00:00,2023-12-15 23:50:00


In [4]:
agg_sum.to_csv("Flow_depth.csv",index=False)

In [5]:
agg_sum.shape

(8, 26)

In [14]:
df = pd.read_excel("Flow_rate.xlsx")
df.head()

,IDs:,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100190,92100220,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1
0,Date/Time,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,...,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow,Total inflow
1,M/d/yyyy,cfs,cfs,cfs,cfs,cfs,cfs,cfs,cfs,cfs,...,cfs,cfs,cfs,cfs,cfs,cfs,cfs,cfs,cfs,cfs
2,2023-10-01 00:10:00,0.029362,0.012352,0.017731,0.010798,0.030601,0.022105,0.006998,0.010816,0.007112,...,0.002225,0.025429,0.024921,0.033699,0.013563,0.014226,0.031361,0.051982,0.030312,0.01432
3,2023-10-01 00:20:00,0.055978,0.012352,0.04493,0.010798,0.059008,0.057283,0.006998,0.01969,0.007112,...,0.002136,0.06274,0.06432,0.05087,0.054391,0.042528,0.050464,0.055739,0.030312,0.040238
4,2023-10-01 00:30:00,0.095967,0.012352,0.086999,0.010798,0.105134,0.098656,0.006998,0.025159,0.007112,...,0.002134,0.097285,0.100896,0.077127,0.089243,0.06186,0.078598,0.055672,0.030312,0.077798


In [15]:
df = df.iloc[2:].reset_index(drop=True)
df = df.rename(columns = {'IDs:':'Date/Time'})
df.head()

,Date/Time,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100190,92100220,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1
0,2023-10-01 00:10:00,0.029362,0.012352,0.017731,0.010798,0.030601,0.022105,0.006998,0.010816,0.007112,...,0.002225,0.025429,0.024921,0.033699,0.013563,0.014226,0.031361,0.051982,0.030312,0.01432
1,2023-10-01 00:20:00,0.055978,0.012352,0.04493,0.010798,0.059008,0.057283,0.006998,0.01969,0.007112,...,0.002136,0.06274,0.06432,0.05087,0.054391,0.042528,0.050464,0.055739,0.030312,0.040238
2,2023-10-01 00:30:00,0.095967,0.012352,0.086999,0.010798,0.105134,0.098656,0.006998,0.025159,0.007112,...,0.002134,0.097285,0.100896,0.077127,0.089243,0.06186,0.078598,0.055672,0.030312,0.077798
3,2023-10-01 00:40:00,0.142564,0.012352,0.134896,0.010798,0.14484,0.134688,0.006998,0.02557,0.007112,...,0.002134,0.126184,0.136423,0.095605,0.123111,0.063293,0.090512,0.055665,0.030312,0.126097
4,2023-10-01 00:50:00,0.178755,0.012352,0.166028,0.010798,0.17138,0.152587,0.006998,0.025531,0.007112,...,0.002134,0.132118,0.149922,0.096855,0.134079,0.060209,0.089423,0.055664,0.030312,0.168424


In [7]:
import pandas as pd
import numpy as np

# --- assume df is your DataFrame and the datetime column is named 'Date/Time' ---
# If your datetime column has a different name, change it below.
df = df.copy()
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

# create year/month/half indicator (half = 1 for 1..15, 2 for 16..end)
df['year']  = df['Date/Time'].dt.year
df['month'] = df['Date/Time'].dt.month
df['half']  = np.where(df['Date/Time'].dt.day <= 15, 1, 2)

# assign a sequential half-month id (1,2,3...) in chronological order
# groupby(..., sort=True).ngroup() will produce 0-based group numbers in sorted order
df['period_id'] = df.groupby(['year','month','half'], sort=True).ngroup() + 1
df['period'] = (
    df['year'].astype(str) + "-" +
    df['month'].astype(str).str.zfill(2) + "-" +
    df['half'].astype(str)
)

# choose the node columns to aggregate (everything except helper columns)
exclude = ['Date/Time','year','month','half','period_id','period']
node_cols = [c for c in df.columns if c not in exclude]

# --- AGGREGATION OPTION A: SUM (default) ---
agg_sum = df.groupby('period', sort=True)[node_cols].mean().reset_index()

# If you want a meaningful representative datetime for each period, add:
# create a mapping from period -> (period_start, period_end) using the earliest/largest timestamp
period_bounds = (df.groupby('period', sort=True)
                   .agg(period_start=('Date/Time','min'),
                        period_end  =('Date/Time','max'))
                   .reset_index())
agg_sum = agg_sum.merge(period_bounds, on='period')

# --- AGGREGATION OPTION B: MEAN (if you prefer averages instead of sums) ---
agg_mean = df.groupby('period', sort=True)[node_cols].mean().reset_index()
agg_mean = agg_mean.merge(period_bounds, on='period')

agg_sum.head()

,period,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1,period_start,period_end
0,2023-10-1,0.282947,0.020878,0.250353,0.016538,0.248306,0.212762,0.012507,0.042131,0.018214,...,0.209579,0.135889,0.187412,0.08606,0.125015,0.080987,0.042083,0.282612,2023-10-01 00:10:00,2023-10-15 23:50:00
1,2023-10-2,0.219757,0.014065,0.19666,0.011791,0.192625,0.166938,0.007933,0.028637,0.007895,...,0.163737,0.104325,0.146121,0.065487,0.096482,0.061747,0.033262,0.219757,2023-10-16 00:00:00,2023-10-31 23:50:00
2,2023-11-1,0.280108,0.020027,0.248126,0.014298,0.242728,0.211956,0.011128,0.038829,0.010246,...,0.207546,0.133583,0.186764,0.084782,0.123012,0.079768,0.041128,0.280111,2023-11-01 00:00:00,2023-11-15 23:50:00
3,2023-11-2,0.209704,0.01308,0.188074,0.011371,0.184261,0.159426,0.007405,0.026948,0.007505,...,0.156424,0.099447,0.139343,0.06227,0.092055,0.058741,0.031942,0.209705,2023-11-16 00:00:00,2023-11-30 23:50:00
4,2023-12-1,0.255609,0.017631,0.227134,0.013288,0.222073,0.193095,0.009843,0.036294,0.009387,...,0.189126,0.119685,0.169543,0.074794,0.110145,0.070638,0.038484,0.255654,2023-12-01 00:00:00,2023-12-15 23:50:00


In [8]:
agg_sum.shape

(8, 26)

In [9]:
agg_sum.to_csv("Flow_rate.csv",index=False)